# Nate

In [ ]:
# ! pip3 install -r /content/drive/MyDrive/Final\ Project/VRGNet/requirements.txt

# Restormer Data

In [2]:
import os
os.makedirs("../generated_data_restormer/")
os.makedirs("../generated_data_restormer/crop_patch/")
os.makedirs("../generated_data_restormer/fake_rain_layer")
os.makedirs("../generated_data_restormer/rain_fake")
# %pwd

In [1]:
import cv2
import os
import argparse
import glob
import numpy as np
import torch
from torch.autograd import Variable
from vrgnet import EDNet  # EDNet: RNet+G
import time
import matplotlib.pyplot as plt
import random
import scipy.io as sio
from numpy.random import RandomState
from skimage import  exposure
import sys
from tqdm import tqdm


# data_path = "/content/drive/MyDrive/Final Project/VRGNet/input_data/rain"
# gt_path = "/content/drive/MyDrive/Final Project/VRGNet/input_data/no_rain"
data_path = "../../train/Rain13K/input"
gt_path = "../../train/Rain13K/target"
nc = 3 
patch_size = 64
nz = 128
stage = 6
nef = 32
use_gpu = True
gpu_id = "0"

#netEDPath = "./spamodels/ED_state_800.pt"
# netEDPath = '/content/drive/MyDrive/Final Project/VRGNet/for_syn/syn100hmodels/ED_state_700.pt'
# use rain14000 model
netEDPath = "syn1400models/ED_state_700.pt"
save_patch = "../generated_data_restormer/crop_patch/"
save_fake_rain_layer = "../generated_data_restormer/fake_rain_layer"
save_rainfake = "../generated_data_restormer/rain_fake"
num_crops = 5
num_crops = 1

os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id

def is_image(img_name):
    if img_name.endswith(".jpg") or img_name.endswith(".bmp") or img_name.endswith(".png"):
        return True
    else:
        return False

def crop(img, crop_sample):
    h, w, c = img.shape
    p_h, p_w = patch_size, patch_size
    rand_state = RandomState(66*crop_sample)
    r = rand_state.randint(0, h - p_h)
    c = rand_state.randint(0, w - p_w)
    O = img[r: r + p_h, c : c + p_w]
    return O,r,c

print('Loading model ...\n')
netED = EDNet(nc, nz, nef).cuda()
netED.load_state_dict(torch.load(netEDPath))
netED.eval()
z_list=[]
B_list=[]
G_list=[]
image_list = os.listdir(data_path)

# print(len(image_list))
# 13711

for img_name in tqdm(image_list):
    if is_image(img_name):
        img_path = os.path.join(data_path, img_name)
        # gt_img_path = os.path.join(gt_path, "norain" + img_name[4::])
        gt_img_path = os.path.join(gt_path, img_name)

        # print(img_path, gt_img_path)

        # plt.imshow(cv2.imread(img_path))
        # plt.show()

        # plt.imshow(cv2.imread(gt_img_path))
        # plt.show()

        # sys.exit()

        for crop_sample in range(0,num_crops,1):

            # input image
            O = cv2.imread(img_path)
            # plt.imshow(O)
            # plt.show()
            b, g, r = cv2.split(O)
            input_img = cv2.merge([r, g, b])
            # plt.imshow(input_img)
            # plt.show()
            # input_img_test = cv2.imread(img_path)
            # plt.imshow(input_img_test)
            # plt.show()
            # input_img_test_rgb = cv2.cvtColor(input_img_test, cv2.COLOR_BGR2RGB)
            # plt.imshow(input_img_test_rgb)
            # plt.show()

            # crop input to 64x64 and save
            # O, row, col = crop(input_img, crop_sample)
            # resize to 64x64
            O = cv2.resize(O, (64, 64))
            O = O.astype(np.float32) / 255
            O_patch = O
            b, g, r = cv2.split(O)
            O = cv2.merge([r, g, b])
            # plt.imshow(O)
            # cv2_imshow(O)
            # plt.show()
            #cv2.imwrite(os.path.join(save_patch, f"{crop_sample}" + img_name), np.uint8(255 * O))
            # plt.imsave("/content/pixelated.jpg", O)

            

            # crop gt image to 64*64
            B = cv2.imread(gt_img_path)
            #resize to 64x64
            # print(gt_img_path)
            b, g, r = cv2.split(B)
            gt_img = cv2.merge([r, g, b])
            # B = gt_img[row: row + patch_size, col: col + patch_size]
            B = cv2.resize(B, (64, 64))
            B = B.astype(np.float32) / 255
            B_patch = B

            # plt.imshow(B)
            # cv2_imshow(B)
            # plt.show()

            # B_patch = B
            # save the original rain patch
            Rain_patch = O_patch-B_patch
            b, g, r = cv2.split(Rain_patch)
            Rain_patch = cv2.merge([r, g, b])
            # cv2.imwrite((save_patch + '/'+  f"{crop_sample}" + img_name),np.uint8(255*B_patch))

            # plt.imshow(Rain_patch)
            # plt.show()

            # execute latent space interpolation for patch image
            O = np.expand_dims(O_patch.transpose(2, 0, 1), 0)
            O = Variable(torch.Tensor(O)).cuda()
            B = np.expand_dims(B_patch.transpose(2, 0, 1), 0)
            B = torch.Tensor(B).cuda()
            B_list.append(B)
            with torch.no_grad():  #
                if use_gpu:
                    torch.cuda.synchronize()
                _, _, _,z = netED(O) # z: 1*nz
                z_list.append(z)

            # break
    # break

print(len(B_list))
print(len(z_list))

# sys.exit()

for idx, img_name in enumerate(tqdm(image_list)):

    # idx = idx*5
    idx = idx*num_crops
    
    for crop_sample in range(0,num_crops,1):
        # print(idx+crop_sample)

        z_mix = z_list[idx+crop_sample]
        rain_fake = netED.sample(z_mix)
        rain_fake_max = torch.max(rain_fake,1)[0]
        rain_fake = rain_fake_max.unsqueeze(dim=1).expand_as(rain_fake) #gray rain layer


        fake = B_list[idx+crop_sample] + rain_fake
        fake = torch.clamp(fake, 0, 1)
        fake = np.uint8(255 * fake.data.cpu().numpy().squeeze()).transpose(1, 2, 0)

        # plt.imshow(cv2.cvtColor(fake, cv2.COLOR_BGR2RGB))
        # plt.show()

        # adjust brightness
        rain_fake = torch.clamp(rain_fake, 0, 1)
        rain_fake = np.uint8(255*rain_fake.data.cpu().numpy().squeeze()).transpose(1, 2, 0)
        rain_fake = exposure.adjust_gamma(rain_fake,0.7)

        # plt.imshow(rain_fake)
        # plt.show()

        # bg label
        label = torch.clamp(B_list[idx+crop_sample], 0, 1)
        label = np.uint8(255*label.data.cpu().numpy().squeeze()).transpose(1, 2, 0)

        # plt.imshow(label)
        # plt.show()

        img_name = img_name[:img_name.rfind('.')] + '.png'

        #rain-1.png
        # print(save_rainfake + '/'+ f"rain-" + img_name)
        plt.imsave(save_rainfake + '/'+ f"rain-" + img_name, fake[:,:,::-1]/255) # rainy image

        #norain-1.png
        # print(save_patch + '/'+ f"norain-" + img_name)
        plt.imsave(save_patch + '/'+ f"norain-" + img_name, label[:,:,::-1]/255)


        #rainstreak-1.png
        # print(save_fake_rain_layer + '/'+ f"rainstreak-" + img_name)
        plt.imsave(save_fake_rain_layer + '/' + f"rainstreak-" + img_name, rain_fake[:,:,::-1]/255) 

        # print(save_rainfake + '/'+ f"fake_" + f"{crop_sample}" + img_name)

        # plt.imsave(save_rainfake + '/'+ f"fake_" + f"{crop_sample}" + img_name, fake/255) # rainy image
        # plt.imsave(save_fake_rain_layer + '/' + f"fake_" + f"{crop_sample}" + img_name, rain_fake/255)

        # break

    # break



Loading model ...



 26%|█████████████████████████████████████████████▏                                                                                                                              | 3599/13711 [00:35<01:38, 102.64it/s]


KeyboardInterrupt: 